In [1]:
#Importing Packages
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.preprocessing as pre
from sklearn.preprocessing import StandardScaler
import os
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import LabelEncoder


In [2]:
#Move to CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
# #Importing data from UNSW_NB15
# train_df = pd.read_csv('data/UNSW_NB15_training-set.csv')
# test_df = pd.read_csv('data/UNSW_NB15_testing-set.csv')
#https://www.kaggle.com/datasets/nikhil1e9/loan-default
df = pd.read_csv('data/Loan_default.csv')

In [4]:
object_cols = ['Education', 'EmploymentType', 'MaritalStatus', 'HasMortgage', 'HasDependents', 'LoanPurpose', 'HasCoSigner']

# Initialize LabelEncoder
le = LabelEncoder()

# Apply Label Encoding to each object column
for col in object_cols:
    df[col] = le.fit_transform(df[col])

train_df, test_df = tts(df)


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 191510 entries, 64629 to 184431
Data columns (total 18 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   LoanID          191510 non-null  object 
 1   Age             191510 non-null  int64  
 2   Income          191510 non-null  int64  
 3   LoanAmount      191510 non-null  int64  
 4   CreditScore     191510 non-null  int64  
 5   MonthsEmployed  191510 non-null  int64  
 6   NumCreditLines  191510 non-null  int64  
 7   InterestRate    191510 non-null  float64
 8   LoanTerm        191510 non-null  int64  
 9   DTIRatio        191510 non-null  float64
 10  Education       191510 non-null  int64  
 11  EmploymentType  191510 non-null  int64  
 12  MaritalStatus   191510 non-null  int64  
 13  HasMortgage     191510 non-null  int64  
 14  HasDependents   191510 non-null  int64  
 15  LoanPurpose     191510 non-null  int64  
 16  HasCoSigner     191510 non-null  int64  
 17  Default    

In [6]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63837 entries, 31069 to 202460
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   LoanID          63837 non-null  object 
 1   Age             63837 non-null  int64  
 2   Income          63837 non-null  int64  
 3   LoanAmount      63837 non-null  int64  
 4   CreditScore     63837 non-null  int64  
 5   MonthsEmployed  63837 non-null  int64  
 6   NumCreditLines  63837 non-null  int64  
 7   InterestRate    63837 non-null  float64
 8   LoanTerm        63837 non-null  int64  
 9   DTIRatio        63837 non-null  float64
 10  Education       63837 non-null  int64  
 11  EmploymentType  63837 non-null  int64  
 12  MaritalStatus   63837 non-null  int64  
 13  HasMortgage     63837 non-null  int64  
 14  HasDependents   63837 non-null  int64  
 15  LoanPurpose     63837 non-null  int64  
 16  HasCoSigner     63837 non-null  int64  
 17  Default         63837 non-null 

In [7]:
#Collecting required features
feature_columns = ['Age', 'Income', 'LoanAmount', 'CreditScore', 'MonthsEmployed', 'NumCreditLines', 'InterestRate', 'LoanTerm', 'DTIRatio', 'Education', 'EmploymentType', 'MaritalStatus', 'HasMortgage', 'HasDependents', 'LoanPurpose', 'HasCoSigner']# 'attack_cat_encoded'

X_train = train_df[feature_columns].values
y_train = train_df['Default'].values

#
# le = pre.LabelEncoder()
# train_df['attack_cat_encoded'] = le.fit_transform(train_df['attack_cat'])

X_test = test_df[feature_columns].values
y_test = test_df['Default'].values

In [8]:
#Converting to tensor
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Converting to tensor and moving to the selected device

X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).unsqueeze(1).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).unsqueeze(1).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

In [9]:
BATCH_SIZE = 64
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print("Data loaded from CSV and prepared for PyTorch.")
print("Training data shape:", X_train_tensor.shape)
print("Test data shape:", X_test_tensor.shape)
print(f"Number of unique labels (classes): {len(np.unique(y_train))}")


Data loaded from CSV and prepared for PyTorch.
Training data shape: torch.Size([191510, 1, 16])
Test data shape: torch.Size([63837, 1, 16])
Number of unique labels (classes): 2


In [10]:
class Net1DCNN(nn.Module):
    def __init__(self, input_channels, sequence_length, num_classes):
        super(Net1DCNN, self).__init__()

        self.conv1 = nn.Conv1d(in_channels=input_channels, out_channels=32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)


        # self.fc1 = nn.Linear(160 * 1, 64)
        # self.fc2 = nn.Linear(64, num_classes)

        with torch.no_grad():
            dummy_input = torch.randn(1, input_channels, sequence_length)
            dummy_output = self.pool1(self.relu1(self.conv1(dummy_input)))
            dummy_output = self.pool2(self.relu2(self.conv2(dummy_output)))
            flattened_size = dummy_output.view(dummy_output.size(0), -1).shape[1]

        self.fc1 = nn.Linear(flattened_size, 128)
        self.relu_fc = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu_fc(x)
        x = self.fc2(x)
        return x


In [12]:
input_channels = X_train_tensor.shape[1]
sequence_length = X_train_tensor.shape[2]
num_classes = len(np.unique(y_train))

model = Net1DCNN(input_channels=input_channels, sequence_length=sequence_length, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()

#Optimizers
# optimizer = optim.Adam(model.parameters(), lr=0.0005)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.85)

num_epochs = 50

print("\nStarting model training...")
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    train_accuracy = 100 * correct / total

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Train Accuracy: {train_accuracy:.2f}%")

print("Training finished.")


Starting model training...


KeyboardInterrupt: 

In [55]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"\nAccuracy on the test set: {accuracy:.2f}%")


Accuracy on the test set: 84.30%


In [ ]:
model_dir = 'models'
os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, '1d_cnn_model.pth')


torch.save(model.state_dict(), model_path)
print(f"\nModel saved to {model_path}")